In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings  # Updated import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage, HumanMessage
import os
import numpy as np

In [2]:
memory = MemorySaver()
api_key = os.getenv("OPENAI_API_KEY_NUS")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key)

In [3]:
# Load the PDF document
loader = PyPDFLoader(file_path="./data/CERT_RAG.pdf")
docs = loader.load()

# Split the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
# Create the vector store using FAISS
vectorstore = FAISS.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

# Set up the retriever
retriever = vectorstore.as_retriever()

In [5]:
### Build retriever tool ###
tool = create_retriever_tool(
    retriever,
    "pdf_retriever",
    "Searches and returns excerpts from the uploaded PDF document.",
)
tools = [tool]

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [9]:
def read_question_from_txt(file_path):
    # Read the contents of the .txt file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Extract the question and choices
    question = lines[0].strip()  # First line is the question
    choice_a = lines[1].strip()  # Second line is choice A
    choice_b = lines[2].strip()  # Third line is choice B
    choice_c = lines[3].strip()  # Fourth line is choice C
    choice_d = lines[4].strip()  # Fifth line is choice D

    # Format the query string
    query = f"{question}\n (a) {choice_a}\n (b) {choice_b}\n (c) {choice_c}\n (d) {choice_d}"
    return query

### Run this

In [17]:
file_path = './rag-input.txt'
query = read_question_from_txt(file_path)

config = {"configurable": {"thread_id": "abc123"}}

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

whaat is one way to understand current WIP in a system?
 (a) pair to complete the work faster
 (b) make current work visible
 (c) split stories
 (d) size stories smaller
================================== Ai Message ==================================
Tool Calls:
  pdf_retriever (call_dwR1IA0OSbiI4cPP2kjiFtU1)
 Call ID: call_dwR1IA0OSbiI4cPP2kjiFtU1
  Args:
    query: understand current WIP in a system
================================= Tool Message =================================
Name: pdf_retriever

The first corrective action is to make the current WIP visible to all stakeholders. Figure 2 shows a
simple Kanban board that illustrates the total amount of WIP and the process state of each work
item. This Kanban serves as an initial process diagnostic, showing the current bottlenecks. Often,
simply visualizing the current volume of work is the wake-up call that causes the organization to
address the syste